In [ ]:
%%HTML
<style>
    body {
        --vscode-font-family: "Inter";
    }
</style>


<center>
<h1 style="font-size: 50px; font-weight: bold; color:sandybrown">OC SWITRS Data Processing</h1>

<div style="font-size: 40px; font-weight: bold; color: sandybrown">Part 04: ArcGIS Hotspot Analysis</div>
<div style="font-size: 30px; font-weight: bold; color: sandybrown">v.1, November 2024</div>
</center>

---

## <font color="orangered">**Referencing Libraries and Initialization**</font>

### <font color="lime">**Preliminaries**</font>

Instantiating python libraries for the project

In [3]:
# Import Python libraries
import os, json, pytz, math, arcpy, arcgis
from datetime import date, time, datetime, timedelta, tzinfo, timezone
from tqdm.notebook import trange, tqdm, tqdm_notebook
import pandas as pd
import numpy as np
from arcpy import metadata as md

# important as it "enhances" Pandas by importing these classes (from ArcGIS API for Python)
from arcgis.features import GeoAccessor, GeoSeriesAccessor


### <font color="lime">**Project and Workspace Variables**</font>

Define and maintain project, workspace, ArcGIS, and data-related variables.

Project and ArcGIS Pro project paths

In [4]:
# Environment variables for OneDrive path
#oneDrivePath = os.getenv("OneDriveCommercial")

# OC SWITRS project path
#projectPath = os.path.join(oneDrivePath, "Projects", "OCTraffic", "OCSWITRS")
projectPath = os.path.join(r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS")

# OC SWITRS Hotspot Analysis ArcGIS Pro project path
agpDataPath = os.path.join(projectPath, "AGPSWITRS")
agpAnalysisPath = os.path.join(projectPath, "AGPSWITRS_HSA")



ArcGIS Pro-related paths

In [5]:
# ArcGIS Pro project name and path
aprxData = "AGPSWITRS.aprx"
aprxAnalysis = "AGPSWITRS_HSA.aprx"
aprxDataPath = os.path.join(agpDataPath, aprxData)
aprxAnalysisPath = os.path.join(agpAnalysisPath, aprxAnalysis)

# ArcGIS Pro project geodatabase and path
gdbData = "AGPSWITRS.gdb"
gdbAnalysis = "OCSWITRS_HSA.gdb"
gdbDataPath = os.path.join(agpDataPath, gdbData)
gdbAnalysisPath = os.path.join(agpAnalysisPath, gdbAnalysis)

# USC 2020 geodatabase path
gdbUSC2020Path = os.path.join(r"I:\Professional\Projects-OCPW\OCGD\USC2020\USC2020.gdb")

# Current ArcGIS workspace (arcpy)
arcpy.env.workspace = gdbAnalysisPath
workspace = arcpy.env.workspace

# Enable overwriting existing outputs
arcpy.env.overwriteOutput = True


Project folder paths

In [6]:
# Raw data folder path
rawDataPath = os.path.join(projectPath, "RawData")

# Layers folder path
layersPath = os.path.join(projectPath, "Layers")

# Notebooks folder path
notebooksPath = os.path.join(projectPath, "Notebooks")

# Supporting data path on the project geodatabase (feature directory)
supportingDataPath = os.path.join(projectPath, "SupportingData")


ArcGIS Pro folder and geodatabase and data paths

In [ ]:
# Define the arcpy environment for the data geodatabase
arcpy.env.workspace = gdbDataPath
arcpy.env.overwriteOutput = True

# List all feature classes in the geodatabase root
gdbDataFCList = arcpy.ListFeatureClasses(feature_type='Point')
print(f"Geodatabase Point Feature Classes: {gdbDataFCList}")

# Get the feature classes from the data geodatabase root
crashesPoints = arcpy.ListFeatureClasses("Crashes*")[0]
partiesPoints = arcpy.ListFeatureClasses("Parties*")[0]
victimsPoints = arcpy.ListFeatureClasses("Victims*")[0]
collisionsPoints = arcpy.ListFeatureClasses("Collisions*")[0]

# Get the paths of each of the layers: Crashes, Parties, Victims, Collisions
crashesPointsPath = os.path.join(gdbDataPath, crashesPoints)
partiesPointsPath = os.path.join(gdbDataPath, partiesPoints)
victimsPointsPath = os.path.join(gdbDataPath, victimsPoints)
collisionsPointsPath = os.path.join(gdbDataPath, collisionsPoints)

# Get the path for the USC 2020 Blocks data
usc2020blkPath = os.path.join(gdbUSC2020Path, "USC2020_BLK")

# Supporting data pathsboundariesPath = os.path.join(gdbDataPath, ")
boundariesPath = os.path.join(gdbDataPath, "SupportingData", "OCSWITRS_Boundaries")
citiesPath = os.path.join(gdbDataPath, "SupportingData", "OCSWITRS_Cities")
roadsPath = os.path.join(gdbDataPath, "SupportingData", "OCSWITRS_Roads")


print(f"\nFeature Paths:\n- Crashes: {crashesPointsPath}\n- Parties: {partiesPointsPath}\n- Victims: {victimsPointsPath}\n- Collisions: {collisionsPointsPath}\n")
print(f"\nUSC 2020:\n- Census Blocks: {usc2020blkPath}\n")
print(f"\nSupporting Data Paths:\n- Boundaries: {boundariesPath}\n- Cities: {citiesPath}\n- Roads: {roadsPath}\n")


In [8]:
# Current ArcGIS Pro project
aprx = arcpy.mp.ArcGISProject(aprxAnalysisPath)

# Current ArcGIS Pro project map
map = aprx.listMaps()[0]


In [ ]:
# Add the four feature classes (Crashes Points, Parties Points, Victims Points, Collisions Points) from the geodatabase to the map
crashesLyrName = map.addDataFromPath(crashesPointsPath)
partiesLyrName =map.addDataFromPath(partiesPointsPath)
victimsLyrName = map.addDataFromPath(victimsPointsPath)
collisionsLyrName = map.addDataFromPath(collisionsPointsPath)

# Add the Census 2020 blocks to the map
usc2020blkLyrName = map.addDataFromPath(usc2020blkPath)

# Add the supporting data Cities and Roads layers to the map
citiesLyrName = map.addDataFromPath(citiesPath)
roadsLyrName = map.addDataFromPath(roadsPath)

print(f"Added layers to the map:\n\t- {crashesLyrName}\n\t- {partiesLyrName}\n\t- {victimsLyrName}\n\t- {collisionsLyrName}\n\t- {usc2020blkLyrName}\n\t- {citiesLyrName}\n\t- {roadsLyrName}")


Create a summary for each of the Census blocks that contains statistics and counts of crash collision data

In [ ]:
arcpy.analysis.SummarizeWithin(
    in_polygons="USC2020_BLK",
    in_sum_features="OCSWITRS Crashes",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\USC2020_BLK_SumCrashes",
    keep_all_polygons="KEEP_ALL",
    sum_fields="VICTIM_COUNT Sum;NUMBER_KILLED Sum;NUMBER_INJURED Sum;COUNT_SEVERE_INJ Sum;COUNT_VISIBLE_INJ Sum;COUNT_COMPLAINT_PAIN Sum;COUNT_PED_KILLED Sum;COUNT_PED_INJURED Sum;COUNT_BICYCLIST_KILLED Sum;COUNT_BICYCLIST_INJURED Sum;COUNT_MC_KILLED Sum;COUNT_MC_INJURED Sum;COLLISION_SEVERITY_RANKED Mean;COLLISION_SEVERITY_RECLASS Mean",
    sum_shape="ADD_SHAPE_SUM",
    shape_unit="SQUAREKILOMETERS",
    group_field=None,
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="NO_PERCENT",
    out_group_table=None
)


Create a summary for each of the cities that contains statistics and counts of crash collision data

In [ ]:
arcpy.analysis.SummarizeWithin(
    in_polygons="OCSWITRS Cities",
    in_sum_features="OCSWITRS Crashes",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\Cities_SumCrashes",
    keep_all_polygons="KEEP_ALL",
    sum_fields="VICTIM_COUNT Sum;NUMBER_KILLED Sum;NUMBER_INJURED Sum;COUNT_SEVERE_INJ Sum;COUNT_VISIBLE_INJ Sum;COUNT_COMPLAINT_PAIN Sum;COUNT_PED_KILLED Sum;COUNT_PED_INJURED Sum;COUNT_BICYCLIST_KILLED Sum;COUNT_BICYCLIST_INJURED Sum;COUNT_MC_KILLED Sum;COUNT_MC_INJURED Sum;COLLISION_SEVERITY_RANKED Mean;COLLISION_SEVERITY_RECLASS Mean",
    sum_shape="ADD_SHAPE_SUM",
    shape_unit="SQUAREKILOMETERS",
    group_field=None,
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="NO_PERCENT",
    out_group_table=None
)


In [ ]:
arcpy.analysis.SummarizeWithin(
    in_polygons="MajorRoadBuffers",
    in_sum_features="OCSWITRS Crashes",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\MajorRoadBuffers_SumCrashes",
    keep_all_polygons="KEEP_ALL",
    sum_fields="VICTIM_COUNT Sum;NUMBER_KILLED Sum;NUMBER_INJURED Sum;COUNT_SEVERE_INJ Sum;COUNT_VISIBLE_INJ Sum;COUNT_COMPLAINT_PAIN Sum;COUNT_PED_KILLED Sum;COUNT_PED_INJURED Sum;COUNT_BICYCLIST_KILLED Sum;COUNT_BICYCLIST_INJURED Sum;COUNT_MC_KILLED Sum;COUNT_MC_INJURED Sum;COLLISION_SEVERITY_RANKED Mean;COLLISION_SEVERITY_RECLASS Mean",
    sum_shape="ADD_SHAPE_SUM",
    shape_unit="SQUAREKILOMETERS",
    group_field=None,
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="NO_PERCENT",
    out_group_table=None
)


In [ ]:
arcpy.analysis.Select(
    in_features="OCSWITRS Roads",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\MajorRoads",
    where_clause="ROADCAT = 'Primary' Or ROADCAT = 'Secondary'"
)


In [ ]:
arcpy.analysis.Buffer(
    in_features="MajorRoads",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\MajorRoadBuffers",
    buffer_distance_or_field="250 Meters",
    line_side="FULL",
    line_end_type="FLAT",
    dissolve_option="ALL",
    dissolve_field=None,
    method="PLANAR"
)


In [ ]:
arcpy.analysis.SummarizeWithin(
    in_polygons="MajorRoadBuffers",
    in_sum_features="OCSWITRS Crashes",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\MajorRoadBuffers_SummarizeWithin",
    keep_all_polygons="KEEP_ALL",
    sum_fields="VICTIM_COUNT Sum;NUMBER_KILLED Sum;NUMBER_INJURED Sum;COUNT_SEVERE_INJ Sum;COUNT_VISIBLE_INJ Sum;COUNT_COMPLAINT_PAIN Sum;COUNT_PED_KILLED Sum;COUNT_PED_INJURED Sum;COUNT_BICYCLIST_KILLED Sum;COUNT_BICYCLIST_INJURED Sum;COUNT_MC_KILLED Sum;COUNT_MC_INJURED Sum;COLLISION_SEVERITY_RANKED Mean;COLLISION_SEVERITY_RECLASS Mean",
    sum_shape="ADD_SHAPE_SUM",
    shape_unit="SQUAREKILOMETERS",
    group_field=None,
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="NO_PERCENT",
    out_group_table=None
)


In [ ]:
arcpy.management.GeneratePointsAlongLines(
    Input_Features="MajorRoads",
    Output_Feature_Class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\MajorRoads_GeneratePointsAlongLines",
    Point_Placement="DISTANCE",
    Distance="1000 Feet",
    Percentage=None,
    Include_End_Points="NO_END_POINTS",
    Add_Chainage_Fields="NO_CHAINAGE",
    Distance_Field=None,
    Distance_Method="PLANAR"
)


In [ ]:
arcpy.management.SplitLineAtPoint(
    in_features="MajorRoads",
    point_features="MajorRoads_GeneratePointsAlongLines",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\MajorRoadsSplit",
    search_radius="1000 Feet"
)


In [ ]:
arcpy.analysis.Buffer(
    in_features="MajorRoadsSplit",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\MajorRoadsSplitBuffer",
    buffer_distance_or_field="500 Feet",
    line_side="FULL",
    line_end_type="FLAT",
    dissolve_option="NONE",
    dissolve_field=None,
    method="PLANAR"
)


In [ ]:
arcpy.analysis.SummarizeWithin(
    in_polygons="MajorRoadsSplitBuffer",
    in_sum_features="OCSWITRS Crashes",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\MajorRoadsSplitBuffer_SumCrashes",
    keep_all_polygons="KEEP_ALL",
    sum_fields="VICTIM_COUNT Sum;NUMBER_KILLED Sum;NUMBER_INJURED Sum;COUNT_SEVERE_INJ Sum;COUNT_VISIBLE_INJ Sum;COUNT_COMPLAINT_PAIN Sum;COUNT_PED_KILLED Sum;COUNT_PED_INJURED Sum;COUNT_BICYCLIST_KILLED Sum;COUNT_BICYCLIST_INJURED Sum;COUNT_MC_KILLED Sum;COUNT_MC_INJURED Sum;COLLISION_SEVERITY_RANKED Mean;COLLISION_SEVERITY_RECLASS Mean",
    sum_shape="ADD_SHAPE_SUM",
    shape_unit="SQUAREKILOMETERS",
    group_field=None,
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="NO_PERCENT",
    out_group_table=None
)


In [ ]:
arcpy.stats.HotSpots(
    Input_Feature_Class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\CrashesPoints",
    Input_Field="COLLISION_SEVERITY_RECLASS",
    Output_Feature_Class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\Collisions20132024_HotSpots",
    Conceptualization_of_Spatial_Relationships="FIXED_DISTANCE_BAND",
    Distance_Method="EUCLIDEAN_DISTANCE",
    Standardization="ROW",
    Distance_Band_or_Threshold_Distance=None,
    Self_Potential_Field=None,
    Weights_Matrix_File=None,
    Apply_False_Discovery_Rate__FDR__Correction="NO_FDR",
    number_of_neighbors=None
)


In [ ]:
arcpy.stats.OptimizedHotSpotAnalysis(
    Input_Features="Crashes (2013-2024)",
    Output_Features=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\Crashes20132024_OptimizedHotSpotAnalysis",
    Analysis_Field="COLLISION_SEVERITY_RECLASS",
    Incident_Data_Aggregation_Method="COUNT_INCIDENTS_WITHIN_FISHNET_POLYGONS",
    Bounding_Polygons_Defining_Where_Incidents_Are_Possible=None,
    Polygons_For_Aggregating_Incidents_Into_Counts=None,
    Density_Surface=None,
    Cell_Size=None,
    Distance_Band="1000 Meters"
)


In [ ]:
arcpy.gapro.FindHotSpots(
    point_layer="Crashes (2013-2024)",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\Crashes_FindHotSpots_1000ft_1mi",
    bin_size="1000 Feet",
    neighborhood_size="1 Miles",
    time_step_interval=None,
    time_step_alignment="START_TIME",
    time_step_reference=None
)


In [ ]:
arcpy.gapro.FindHotSpots(
    point_layer="Victims (2013-2024)",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\Victims_FindHotSpots_1000ft_1mi",
    bin_size="1000 Feet",
    neighborhood_size="1 Miles",
    time_step_interval=None,
    time_step_alignment="START_TIME",
    time_step_reference=None
)


In [ ]:
arcpy.gapro.FindHotSpots(
    point_layer="Crashes (2013-2024)",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\Crashes_FindHotSpots_500m_1km",
    bin_size="500 Meters",
    neighborhood_size="1 Kilometers",
    time_step_interval=None,
    time_step_alignment="START_TIME",
    time_step_reference=None
)


In [ ]:
arcpy.gapro.FindHotSpots(
    point_layer="Crashes (2013-2024)",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\Crashes_FindHotSpots_100m_1km",
    bin_size="100 Meters",
    neighborhood_size="1 Kilometers",
    time_step_interval=None,
    time_step_alignment="START_TIME",
    time_step_reference=None
)


In [ ]:
arcpy.gapro.FindHotSpots(
    point_layer="Crashes (2013-2024)",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\Crashes_FindHotSpots_100m_1km",
    bin_size="100 Meters",
    neighborhood_size="1 Kilometers",
    time_step_interval=None,
    time_step_alignment="START_TIME",
    time_step_reference=None
)


In [ ]:
arcpy.gapro.FindHotSpots(
    point_layer="Crashes (2013-2024)",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\Crashes_FindHotSpots_500ft_1mi",
    bin_size="500 Feet",
    neighborhood_size="1 Miles",
    time_step_interval=None,
    time_step_alignment="START_TIME",
    time_step_reference=None
)


In [ ]:
arcpy.stats.HotSpots(
    Input_Feature_Class="Crashes within 500m of Major Roads",
    Input_Field="COLLISION_SEVERITY_RECLASS",
    Output_Feature_Class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\Crasheswithin500mofMajorRoads_HotSpots",
    Conceptualization_of_Spatial_Relationships="FIXED_DISTANCE_BAND",
    Distance_Method="EUCLIDEAN_DISTANCE",
    Standardization="ROW",
    Distance_Band_or_Threshold_Distance=None,
    Self_Potential_Field=None,
    Weights_Matrix_File=None,
    Apply_False_Discovery_Rate__FDR__Correction="NO_FDR",
    number_of_neighbors=None
)


In [ ]:
arcpy.gapro.FindHotSpots(
    point_layer="Crashes within 500m of Major Roads",
    out_feature_class=r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS_HSA\AGPSWITRS_HSA.gdb\Crasheswithin500mofMajorRoads_FindHotSpots_500ft_1mi",
    bin_size="500 Feet",
    neighborhood_size="1 Miles",
    time_step_interval=None,
    time_step_alignment="START_TIME",
    time_step_reference=None
)


In [ ]:
arcpy.stats.ExploratoryRegression(
    Input_Features="Crashes (2013-2024)",
    Dependent_Variable="COLLISION_SEVERITY_BINARY",
    Candidate_Explanatory_Variables="COLLISION_TIME_RUSH_HOURS;NUMBER_KILLED;COUNT_SEVERE_INJ;COUNT_PED_KILLED;COUNT_BICYCLIST_KILLED;COUNT_MC_KILLED;PCF_VIOLATION;CITY_POPDENS;CITY_HUDENS;CITY_POPA;CITY_POPB;CITY_POPH;CITY_POPW",
    Weights_Matrix_File=None,
    Output_Report_File=None,
    Output_Results_Table=None,
    Maximum_Number_of_Explanatory_Variables=5,
    Minimum_Number_of_Explanatory_Variables=1,
    Minimum_Acceptable_Adj_R_Squared=0.5,
    Maximum_Coefficient_p_value_Cutoff=0.05,
    Maximum_VIF_Value_Cutoff=7.5,
    Minimum_Acceptable_Jarque_Bera_p_value=0.1,
    Minimum_Acceptable_Spatial_Autocorrelation_p_value=0.1
)


In [ ]:
arcpy.stats.ExploratoryRegression(
    Input_Features="Collisions (2013-2024)",
    Dependent_Variable="COLLISION_SEVERITY_RECLASS",
    Candidate_Explanatory_Variables="COLLISION_TIME_RUSH_HOURS;WEATHER_COMBINED;VEHICLE_YEAR;VICTIM_ROLE;VICTIM_AGE;CITY_POPDENS;CITY_HUDENS;CITY_POPA;CITY_POPB;CITY_POPH;CITY_POPW",
    Weights_Matrix_File=None,
    Output_Report_File=None,
    Output_Results_Table=None,
    Maximum_Number_of_Explanatory_Variables=5,
    Minimum_Number_of_Explanatory_Variables=1,
    Minimum_Acceptable_Adj_R_Squared=0.5,
    Maximum_Coefficient_p_value_Cutoff=0.05,
    Maximum_VIF_Value_Cutoff=7.5,
    Minimum_Acceptable_Jarque_Bera_p_value=0.1,
    Minimum_Acceptable_Spatial_Autocorrelation_p_value=0.1
)
